# Tool Chain

If you want to use a Tool outside of an Agent, `ToolChain` lets you wrap a chain around your tool:

In [1]:
from langchain_contrib.chains.tool import ToolChain
from langchain_contrib.tools import TerminalTool

terminal = TerminalTool()
terminal_chain = ToolChain(tool=terminal)
terminal_chain({"action_input": "pwd"}, return_only_outputs=True)

{'action_result': '/home/amos/projects/gpt-experiments/langchain-contrib/docs/chains\n'}

You can now interact with this chain from other chains. Imagine that you have a chain to grab SQL records given the current time:

In [2]:
from langchain_contrib.chains.testing import FakeChain

sql_chain = FakeChain(
    expected_inputs=["time"],
    output={"events_from_last_24_hrs": "a,b,c"},
)

It expects a `time` input argument, so you update the output key of the `ToolChain` wrapper:

In [3]:
terminal_chain.tool_output_key = "time"

and chain the two together:

In [4]:
from langchain.chains.sequential import SequentialChain

chained = SequentialChain(
    chains=[terminal_chain, sql_chain],
    input_variables=["action_input"],
    output_variables=["time", "events_from_last_24_hrs"],
    return_all=True,
)
chained({"action_input": "date"}, return_only_outputs=True)

{'time': 'Thu Mar 23 15:03:04 AEDT 2023\n', 'events_from_last_24_hrs': 'a,b,c'}